# Import Library

In [1]:
# Import the numpy and pandas package
import numpy as np
import pandas as pd

# Import plotly packages
import matplotlib.pyplot as plt
import plotly.express as px


from sklearn.preprocessing import MinMaxScaler 

# Import Dataset

In [2]:
dataset = pd.read_excel(r'Dataset_Obat.xlsx')

In [3]:
# Convert Dataset into DataFrame
dataset = pd.DataFrame(dataset)

# EDA

In [4]:
dataset.head()

,Month,Use
0,2015-01-01,12748
1,2015-02-01,13958
2,2015-03-01,12857
3,2015-04-01,13486
4,2015-05-01,13064


In [5]:
#Check Shape
dataset.shape

(84, 2)

In [6]:
#Check Null
dataset.isnull().sum()

Month    0
Use      0
dtype: int64

In [7]:
# Check DataTypes
dataset.dtypes

Month    datetime64[ns]
Use               int64
dtype: object

In [8]:
# Descriptive statistics summary
dataset.describe()

,Use
count,84.000000
mean,13291.166667
std,663.332420
min,12007.000000
25%,12776.000000
50%,13210.500000
75%,13866.750000
max,14589.000000


In [9]:
# Using plotly.express
fig = px.line(dataset, x='Month', y='Use')
fig.show()

In [10]:
# Data distribution
fig = px.histogram(dataset, x='Use', marginal="box", width=700, height=600)
fig.show()

# Data Preprocessing

In [11]:
dataset.index = pd.to_datetime(dataset['Month'])

dataset.drop('Month', axis=1, inplace=True)

dataset.head()

,Use
Month,
2015-01-01,12748
2015-02-01,13958
2015-03-01,12857
2015-04-01,13486
2015-05-01,13064


In [12]:
# MinMaxScaler Normalization

scaler = MinMaxScaler(feature_range=(0, 1))
dataset[['Use']] = scaler.fit_transform(dataset[['Use']])

dataset.head()

,Use
Month,
2015-01-01,0.286987
2015-02-01,0.755616
2015-03-01,0.329202
2015-04-01,0.572812
2015-05-01,0.409373


In [13]:
# Using plotly.express

fig = px.line(dataset, x=dataset.index, y='Use')
fig.show()

In [14]:
# Splitting data
split_time = int(0.8*84) #Training 80% of total index

time_train = dataset.index[:split_time]
time_valid =  dataset.index[split_time:]

#Paracetamol
x_train =  dataset['Use'][:split_time]
x_valid =  dataset['Use'][split_time:]

In [15]:
# Checking data train
print(time_train, x_train, time_valid, x_valid)

DatetimeIndex(['2015-01-01', '2015-02-01', '2015-03-01', '2015-04-01',
               '2015-05-01', '2015-06-01', '2015-07-01', '2015-08-01',
               '2015-09-01', '2015-10-01', '2015-11-01', '2015-12-01',
               '2016-01-01', '2016-02-01', '2016-03-01', '2016-04-01',
               '2016-05-01', '2016-06-01', '2016-07-01', '2016-08-01',
               '2016-09-01', '2016-10-01', '2016-11-01', '2016-12-01',
               '2017-01-01', '2017-02-01', '2017-03-01', '2017-04-01',
               '2017-05-01', '2017-06-01', '2017-07-01', '2017-08-01',
               '2017-09-01', '2017-10-01', '2017-11-01', '2017-12-01',
               '2018-01-01', '2018-02-01', '2018-03-01', '2018-04-01',
               '2018-05-01', '2018-06-01', '2018-07-01', '2018-08-01',
               '2018-09-01', '2018-10-01', '2018-11-01', '2018-12-01',
               '2019-01-01', '2019-02-01', '2019-03-01', '2019-04-01',
               '2019-05-01', '2019-06-01', '2019-07-01', '2019-08-01',
      

In [16]:
# Library tensoflow and keras
import tensorflow as tf
from keras.models import Sequential
from keras.models import model_from_yaml
from keras.layers import Dense
from keras.utils import plot_model

In [17]:
def windowed_dataset(series, window_size, batch_size, shuffle_buffer):
    series = tf.expand_dims(series, axis=-1)
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size + 1, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(window_size + 1))
    ds = ds.shuffle(shuffle_buffer)
    ds = ds.map(lambda w: (w[:-1], w[1:]))
    return ds.batch(batch_size).prefetch(1)

In [18]:
window_size = 2
batch_size = 6
shuffle_buffer_size = 12

#Paracetamol
train_set = windowed_dataset(x_train, window_size, batch_size, shuffle_buffer_size)
valid_set = windowed_dataset(x_valid, window_size, batch_size, shuffle_buffer_size)

# Modelling

## Modelling Paracetamol

In [19]:
model_paracetamol = Sequential()
model_paracetamol.add(Dense(12, input_shape=[1, 1]))
model_paracetamol.add(Dense(6))
model_paracetamol.add(Dense(1))

In [20]:
model_paracetamol.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9),metrics=['mape'],loss='mse')

In [21]:
model_paracetamol.build()
model_paracetamol.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 1, 12)             24        
                                                                 
 dense_1 (Dense)             (None, 1, 6)              78        
                                                                 
 dense_2 (Dense)             (None, 1, 1)              7         
                                                                 
Total params: 109
Trainable params: 109
Non-trainable params: 0
_________________________________________________________________


In [22]:
history_paracetamol = model_paracetamol.fit(train_set,epochs=100,validation_data=valid_set)

Epoch 1/100
11/11 [==============================] - 3s 240ms/step - loss: 0.1598 - mape: 12193184.0000 - val_loss: 0.0604 - val_mape: 50.9363
Epoch 2/100
11/11 [==============================] - 0s 7ms/step - loss: 0.0903 - mape: 7820809.5000 - val_loss: 0.0604 - val_mape: 44.7884
Epoch 3/100
11/11 [==============================] - 0s 7ms/step - loss: 0.0745 - mape: 6967685.0000 - val_loss: 0.0889 - val_mape: 37.4675
Epoch 4/100
11/11 [==============================] - 0s 7ms/step - loss: 0.0627 - mape: 8135748.5000 - val_loss: 0.0640 - val_mape: 41.0993
Epoch 5/100
11/11 [==============================] - 0s 7ms/step - loss: 0.0651 - mape: 7156925.0000 - val_loss: 0.0729 - val_mape: 38.2903
Epoch 6/100
11/11 [==============================] - 0s 7ms/step - loss: 0.0637 - mape: 8234770.5000 - val_loss: 0.0664 - val_mape: 39.9123
Epoch 7/100
11/11 [==============================] - 0s 7ms/step - loss: 0.0645 - mape: 7641428.5000 - val_loss: 0.0661 - val_mape: 40.0340
Epoch 8/100
11/11

In [23]:
model_paracetamol.evaluate(valid_set)

3/3 [==============================] - 0s 13ms/step - loss: 0.0720 - mape: 38.4165


[0.07198452949523926, 38.41645812988281]

In [24]:
import pickle

pickle.dump(history_paracetamol, open("modelParacetamol.pkl", "wb"))

INFO:tensorflow:Assets written to: ram://5340629e-a2ba-46e5-9321-fd46de7e582a/assets


# Save Model

In [28]:
model_yaml = model_paracetamol.to_yaml()
with open ('model.yaml', 'w') as yaml_file:
    yaml_file.write(model_yaml)

model_paracetamol.save_weights('model.h5')
print("save model")

RuntimeError: Method `model.to_yaml()` has been removed due to security risk of arbitrary code execution. Please use `model.to_json()` instead.

In [29]:
img_file = './model_arch.png'

tf.keras.utils.plot_model(model_paracetamol, to_file=img_file, show_shapes=True, show_layer_names=True)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.
